<a href="https://colab.research.google.com/github/yogesh-bhattarai/GEN_AI/blob/main/exercise_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
df= pd.read_csv("/content/ExerciseRecommendation.csv")

In [3]:
df.head(10)

,exercise_id,fitness_goal,exercise_name,exercise_type,equipment_needed,duration_or_reps
0,1,Weight Loss,Jump rope,Cardio,Jump rope,3 sets of 15 reps
1,2,Weight Loss,Burpees,Cardio,NaN,3 sets of 15 reps
2,3,Weight Loss,HIIT,Cardio,Mat,3 sets of 15 reps
3,4,Weight Loss,Running,Cardio,Treadmill / Outdoor,20-40 mins
4,5,Weight Loss,Cycling,Cardio,Stationary bike,20-40 mins
5,6,Weight Loss,Mountain climbers,Cardio,Mat,3 sets of 15 reps
6,7,Weight Loss,Squats,Strength,None / Barbell,3 sets of 8-12 reps
7,8,Muscle Gain,Bench press,Strength,Barbell,3 sets of 8-12 reps
8,9,Muscle Gain,Deadlifts,Strength,Barbell,3 sets of 8-12 reps
9,10,Muscle Gain,Squats,Strength,None / Barbell,3 sets of 8-12 reps


In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

class ExerciseRecommender:
    def __init__(self, df):
        self.df = df.copy()
        self.preprocess()
        self.vectorize()
        self.compute_similarity()

    def preprocess(self):
        # Convert all string columns to lowercase
        str_cols = ["fitness_goal", "exercise_name", "exercise_type", "equipment_needed", "duration_or_reps"]
        for col in str_cols:
            self.df[col] = self.df[col].astype(str).str.lower().str.strip()

        # Combine selected features
        self.df["combined_features"] = self.df.apply(
            lambda row: f"{row['fitness_goal']} {row['exercise_type']} {row['equipment_needed']} {row['duration_or_reps']}",
            axis=1
        )

    def vectorize(self):
        self.vectorizer = TfidfVectorizer()
        self.feature_vectors = self.vectorizer.fit_transform(self.df["combined_features"])

    def compute_similarity(self):
        self.similarity_matrix = cosine_similarity(self.feature_vectors)

    def recommend(self, exercise_name, top_n=5):
        exercise_name = exercise_name.lower().strip()
        if exercise_name not in self.df['exercise_name'].values:
            return "Exercise not found."

        idx = self.df[self.df["exercise_name"] == exercise_name].index[0]
        scores = list(enumerate(self.similarity_matrix[idx]))
        sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
        recommendations = self.df.iloc[[i[0] for i in sorted_scores]]["exercise_name"].values
        return recommendations


In [17]:
# Load the dataset
df = pd.read_csv("/content/ExerciseRecommendation.csv")  # Your dataset path

# Initialize the recommender
recommender = ExerciseRecommender(df)

# Pickle the entire class
with open("exercise_recommender.pkl", "wb") as f:
    pickle.dump(recommender, f)

print("✅ Recommender class pickled successfully.")


✅ Recommender class pickled successfully.


In [20]:
import pickle

with open("exercise_recommender.pkl", "rb") as f:
    recommender = pickle.load(f)

print(recommender.recommend("Jump rope", top_n=1))


['jump rope']



Recommendations for 'Deadlifts' based on fitness_goal, equipment_needed, duration_or_reps:
1. Deadlifts (Type: Strength, Similarity: 1.000)
2. Bench press (Type: Strength, Similarity: 1.000)
